In [3]:
from abc import ABC, abstractmethod
from typing import List, Set
import copy

class Vidnoshennya(ABC):
    def __init__(self, n: int = 0):
        self.n = n
    @abstractmethod
    def is_reflexive(self): pass
    @abstractmethod
    def is_antireflexive(self): pass
    @abstractmethod
    def is_symmetric(self): pass
    @abstractmethod
    def is_asymmetric(self): pass
    @abstractmethod
    def is_antisymmetric(self): pass
    @abstractmethod
    def is_transitive(self): pass
    @abstractmethod
    def is_acyclic(self): pass
    @abstractmethod
    def is_linear(self): pass
    @abstractmethod
    def type_tol(self): pass
    @abstractmethod
    def type_eq(self): pass
    @abstractmethod
    def type_quasi_order(self): pass
    @abstractmethod
    def type_order(self): pass
    @abstractmethod
    def type_strict_order(self): pass
    @abstractmethod
    def type_linear_order(self): pass
    @abstractmethod
    def type_strict_linear_order(self): pass
    @abstractmethod
    def symmetric_part(self): pass
    @abstractmethod
    def asymmetric_part(self): pass
    @abstractmethod
    def transitive_closure(self): pass
    @abstractmethod
    def reachability(self): pass
    @abstractmethod
    def mutual_reachability(self): pass
    @abstractmethod
    def factorize(self, classes): pass


class VidnoshennyaMatr(Vidnoshennya):
    def __init__(self, matrix: List[List[int]]):
        super().__init__(len(matrix))
        self.B = copy.deepcopy(matrix)
    def is_reflexive(self):
        return all(self.B[i][i] == 1 for i in range(self.n))
    def is_antireflexive(self):
        return all(self.B[i][i] == 0 for i in range(self.n))
    def is_symmetric(self):
        return all(self.B[i][j] == self.B[j][i] for i in range(self.n) for j in range(self.n))
    def is_asymmetric(self):
        return self.is_antireflexive() and all(not (self.B[i][j] and self.B[j][i]) for i in range(self.n) for j in range(self.n))
    def is_antisymmetric(self):
        return all(not (self.B[i][j] and self.B[j][i] and i != j) for i in range(self.n) for j in range(self.n))
    def is_transitive(self):
        for i in range(self.n):
            for j in range(self.n):
                if self.B[i][j]:
                    for k in range(self.n):
                        if self.B[j][k] and not self.B[i][k]:
                            return False
        return True
    def is_acyclic(self):
        reach = self.reachability().B
        for i in range(self.n):
            if reach[i][i] == 1:  # цикл
                return False
        return True
    def is_linear(self):
        for i in range(self.n):
            for j in range(self.n):
                if i != j and not (self.B[i][j] or self.B[j][i]):
                    return False
        return True
    def type_tol(self):
        return self.is_reflexive() and self.is_symmetric()
    def type_eq(self):
        return self.is_reflexive() and self.is_symmetric() and self.is_transitive()
    def type_quasi_order(self):
        return self.is_reflexive() and self.is_transitive()
    def type_order(self):
        return self.is_reflexive() and self.is_antisymmetric() and self.is_transitive()
    def type_strict_order(self):
        return self.is_asymmetric() and self.is_transitive()
    def type_linear_order(self):
        return self.type_order() and self.is_linear()
    def type_strict_linear_order(self):
        return self.type_strict_order() and self.is_linear()
    def symmetric_part(self):
        B_inv = self.obernene().B
        newB = [[1 if self.B[i][j] and B_inv[i][j] else 0 for j in range(self.n)] for i in range(self.n)]
        return VidnoshennyaMatr(newB)
    def asymmetric_part(self):
        sym = self.symmetric_part().B
        newB = [[self.B[i][j] if sym[i][j] == 0 else 0 for j in range(self.n)] for i in range(self.n)]
        return VidnoshennyaMatr(newB)
    def transitive_closure(self):
        closure = copy.deepcopy(self.B)
        for k in range(self.n):
            for i in range(self.n):
                for j in range(self.n):
                    closure[i][j] = closure[i][j] or (closure[i][k] and closure[k][j])
        return VidnoshennyaMatr(closure)
    def reachability(self):
        closure = self.transitive_closure().B
        for i in range(self.n):
            closure[i][i] = 1
        return VidnoshennyaMatr(closure)
    def mutual_reachability(self):
        reach = self.reachability()
        reach_inv = reach.obernene()
        return VidnoshennyaMatr([[1 if reach.B[i][j] and reach_inv.B[i][j] else 0 for j in range(self.n)] for i in range(self.n)])
    def factorize(self, classes):
        m = len(classes)
        newB = [[0]*m for _ in range(m)]
        for i in range(m):
            for j in range(m):
                for a in classes[i]:
                    for b in classes[j]:
                        if self.B[a][b] == 1:
                            newB[i][j] = 1
                            break
        return VidnoshennyaMatr(newB)
    def obernene(self):
        return VidnoshennyaMatr([[self.B[j][i] for j in range(self.n)] for i in range(self.n)])
    
    
    
class VidnoshennyaZriz(Vidnoshennya):
    def __init__(self, n: int, zrizy: List[Set[int]]):
        super().__init__(n)
        self.R_plus = copy.deepcopy(zrizy)
    def to_matrix(self):
        B = [[0]*self.n for _ in range(self.n)]
        for i in range(self.n):
            for j in self.R_plus[i]:
                B[i][j] = 1
        return VidnoshennyaMatr(B)
    def is_reflexive(self): return self.to_matrix().is_reflexive()
    def is_antireflexive(self): return self.to_matrix().is_antireflexive()
    def is_symmetric(self): return self.to_matrix().is_symmetric()
    def is_asymmetric(self): return self.to_matrix().is_asymmetric()
    def is_antisymmetric(self): return self.to_matrix().is_antisymmetric()
    def is_transitive(self): return self.to_matrix().is_transitive()
    def is_acyclic(self): return self.to_matrix().is_acyclic()
    def is_linear(self): return self.to_matrix().is_linear()
    def type_tol(self): return self.to_matrix().type_tol()
    def type_eq(self): return self.to_matrix().type_eq()
    def type_quasi_order(self): return self.to_matrix().type_quasi_order()
    def type_order(self): return self.to_matrix().type_order()
    def type_strict_order(self): return self.to_matrix().type_strict_order()
    def type_linear_order(self): return self.to_matrix().type_linear_order()
    def type_strict_linear_order(self): return self.to_matrix().type_strict_linear_order()
    def symmetric_part(self): return self.to_matrix().symmetric_part().to_zriz()
    def asymmetric_part(self): return self.to_matrix().asymmetric_part().to_zriz()
    def transitive_closure(self): return self.to_matrix().transitive_closure().to_zriz()
    def reachability(self): return self.to_matrix().reachability().to_zriz()
    def mutual_reachability(self): return self.to_matrix().mutual_reachability().to_zriz()
    def factorize(self, classes): return self.to_matrix().factorize(classes).to_zriz()
    def to_zriz(self): return self





P_matrix = [
    [1,0,1,1,0],
    [0,0,0,0,0],
    [0,0,1,1,1],
    [0,0,0,0,0],
    [0,0,1,0,0]
]

Q_matrix = [
    [1,1,0,0,0],
    [1,1,0,0,0],
    [0,0,1,1,1],
    [0,0,1,1,1],
    [0,0,1,1,1]
]





P = VidnoshennyaMatr(P_matrix)
Q = VidnoshennyaMatr(Q_matrix)


print("=== Перевірка Q ===")
print("Рефлексивність:", Q.is_reflexive())
print("Антирефлексивність:", Q.is_antireflexive())
print("Симетричність:", Q.is_symmetric())
print("Асиметричність:", Q.is_asymmetric())
print("Антисиметричність:", Q.is_antisymmetric())
print("Транзитивність:", Q.is_transitive())
print("Ациклічність:", Q.is_acyclic())
print("Лінійність:", Q.is_linear())

print("\n=== Тип відношення Q ===")
print("Толерантність:", Q.type_tol())
print("Еквівалентність:", Q.type_eq())
print("Квазіпорядок:", Q.type_quasi_order())
print("Порядок:", Q.type_order())
print("Строгий порядок:", Q.type_strict_order())
print("Лінійний порядок:", Q.type_linear_order())
print("Строгий лінійний порядок:", Q.type_strict_linear_order())


if not Q.is_transitive():
    Q_tc = Q.transitive_closure()
    print("\nQ після транзитивного замикання:")
    for row in Q_tc.B:
        print(row)
    print("\n=== Тип Q після замикання ===")
    print("Толерантність:", Q_tc.type_tol())
    print("Еквівалентність:", Q_tc.type_eq())
    print("Квазіпорядок:", Q_tc.type_quasi_order())
    print("Порядок:", Q_tc.type_order())
    print("Строгий порядок:", Q_tc.type_strict_order())
    print("Лінійний порядок:", Q_tc.type_linear_order())
    print("Строгий лінійний порядок:", Q_tc.type_strict_linear_order())
else:
    print("\nQ вже транзитивне")





print("\n=== Взаємна досягальність P ===")
P_tau = P.mutual_reachability()
for row in P_tau.B:
    print(row)


eq_classes = []
visited = set()
for i in range(P_tau.n):
    if i not in visited:
        cls = [j for j in range(P_tau.n) if P_tau.B[i][j] == 1]
        for j in cls:
            visited.add(j)
        eq_classes.append(cls)

print("\nКласи еквівалентності:", eq_classes)


P_fact = P.factorize(eq_classes)
print("\nМатриця факторизованого відношення:")
for row in P_fact.B:
    print(row)


print("\n=== Перевірка властивостей факторизованого P ===")
print("Рефлексивність:", P_fact.is_reflexive())
print("Антирефлексивність:", P_fact.is_antireflexive())
print("Симетричність:", P_fact.is_symmetric())
print("Асиметричність:", P_fact.is_asymmetric())
print("Антисиметричність:", P_fact.is_antisymmetric())
print("Транзитивність:", P_fact.is_transitive())
print("Ациклічність:", P_fact.is_acyclic())
print("Лінійність:", P_fact.is_linear())

print("\n=== Тип факторизованого P ===")
print("Толерантність:", P_fact.type_tol())
print("Еквівалентність:", P_fact.type_eq())
print("Квазіпорядок:", P_fact.type_quasi_order())
print("Порядок:", P_fact.type_order())
print("Строгий порядок:", P_fact.type_strict_order())
print("Лінійний порядок:", P_fact.type_linear_order())
print("Строгий лінійний порядок:", P_fact.type_strict_linear_order())






def collect_info(obj):
    return {
        "Рефлексивне": obj.is_reflexive(),
        "Антирефлексивне": obj.is_antireflexive(),
        "Симетричне": obj.is_symmetric(),
        "Асиметричне": obj.is_asymmetric(),
        "Антисиметричне": obj.is_antisymmetric(),
        "Транзитивне": obj.is_transitive(),
        "Ациклічне": obj.is_acyclic(),
        "Лінійне": obj.is_linear(),
        "Толерантність": obj.type_tol(),
        "Еквівалентність": obj.type_eq(),
        "Квазіпорядок": obj.type_quasi_order(),
        "Порядок": obj.type_order(),
        "Строгий порядок": obj.type_strict_order(),
        "Лінійний порядок": obj.type_linear_order(),
        "Строгий лінійний порядок": obj.type_strict_linear_order()
    }

print("\n=== Таблиця змін властивостей та типів ===")
info_Q_before = collect_info(Q)
info_Q_after = collect_info(Q)  


info_P_before = collect_info(P)
info_P_after = collect_info(P_fact)


headers = list(info_Q_before.keys())
print(f"{'Властивість/Тип':<25} {'Q (До)':<10} {'Q (Після)':<12} {'P (До)':<10} {'P (Після)':<12}")
for h in headers:
    print(f"{h:<25} {str(info_Q_before[h]):<10} {str(info_Q_after[h]):<12} {str(info_P_before[h]):<10} {str(info_P_after[h]):<12}")

=== Перевірка Q ===
Рефлексивність: True
Антирефлексивність: False
Симетричність: True
Асиметричність: False
Антисиметричність: False
Транзитивність: True
Ациклічність: False
Лінійність: False

=== Тип відношення Q ===
Толерантність: True
Еквівалентність: True
Квазіпорядок: True
Порядок: False
Строгий порядок: False
Лінійний порядок: False
Строгий лінійний порядок: False

Q вже транзитивне

=== Взаємна досягальність P ===
[1, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[0, 0, 1, 0, 1]
[0, 0, 0, 1, 0]
[0, 0, 1, 0, 1]

Класи еквівалентності: [[0], [1], [2, 4], [3]]

Матриця факторизованого відношення:
[1, 0, 1, 1]
[0, 0, 0, 0]
[0, 0, 1, 1]
[0, 0, 0, 0]

=== Перевірка властивостей факторизованого P ===
Рефлексивність: False
Антирефлексивність: False
Симетричність: False
Асиметричність: False
Антисиметричність: True
Транзитивність: True
Ациклічність: False
Лінійність: False

=== Тип факторизованого P ===
Толерантність: False
Еквівалентність: False
Квазіпорядок: False
Порядок: False
Строгий порядок: False
